# Two Tower Model Recommenders System without TFRS Implementation
By TFUG Surabaya Team (Joan Santoso, Billy K., Patrick S.)

### Import Library
Import semua library yang diperlukan. Pada tutorial ini library yang diperlukan adalah tensorflow, numpy, dan tensorflow dataset. <br>
tensorflow digunakan sebagai framework deep learning <br>
numpy merupakan library komputasi matematika di python <br>
tensorflow dataset(tfds) digunakan untuk load dataset yang akan dipakai yaitu MovieLens 100K dataset.
Dataset dapat di download di https://grouplens.org/datasets/movielens/100k/

In [ ]:
!pip install -q --upgrade tensorflow-datasets

In [ ]:
# Import semua library yang diperlukan.
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

### Ambil seluruh data
Kita mengambil seluruh data dengan mengunakan tensorflow dataset, dan juga melakukan preprocessing. Preprocessing yang dilakukan adalah hanya mengambil fitur dasar pada dataset yaitu id dari user dan juga judul dari movie.

In [ ]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# fitur dari semua data movies
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
# Hanya menggunakan judul movi dan user id(tidak menggunakan fitur lain)
# preprocess dengan fungsi map
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Membuat dictionary untuk mengubah user id dan judul movie ke integer yang digunakan untuk layer embedding

In [ ]:
# StringLookup digunakan untuk mengubah string ke index
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
# buat vocabulary dengan StringLookup
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

### Pembuatan Model

Pada tutorial ini kita akan membuat model tanpa reccomender system sederhana tanpa menggunakan library tfrs. Hal ini dapat dilakukan dengan mengunkan framework tensorflow. Model pada bagian ini adalah model yang bersifat general. Model yang kita gunakan adalah twin-tower model dan akan menerima 3 hal yaitu model dari user, model dari movie, dan juga task yang digunakan untuk melatih.

In [ ]:
# Kita menggunakan keras model murni(menurunkan keras model)
class MovieLensModel(tf.keras.Model):
  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task):
    super().__init__()

    # Memasukan user dan movie model ke model utama.
    self.user_model = user_model
    self.movie_model = movie_model

    # menyimpan metrics untuk ditunjukan saat training
    self.my_metrics = []

    # Memasukan task ke dalam model utama
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Menghitung loss function yang telah didefinisikan

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])
    loss,score = self.task(user_embeddings, movie_embeddings)

    indices = tf.argsort(score)

    return loss,score

  def train_step(self, inputs):
    with tf.GradientTape() as tape:
      loss,score = self.compute_loss(inputs, training=True)

      # untuk regularisasi jika ada
      regularization_loss = tf.reduce_sum(
          [tf.reduce_sum(loss) for loss in self.losses]
      )

      total_loss = loss + regularization_loss

    # Update parameter model
    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {}
    # Menghitung metrics evaluasi saat training
    eval_score = tf.argsort(score,direction='DESCENDING').numpy()
    top_ks = [1,5,10,50,100]
    for k_now in top_ks :
      score_now = eval_score[:,0:k_now]
      acc_now = np.array([ (idx in score_now[idx].tolist()) for idx in range(len(score_now))])
      metrics["top_" + str(k_now) + "_categorical_accuracy"] = (acc_now.sum()/len(acc_now))

    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

### Mendefinikan model yang dipakai dan juga task yang digunakan

Model yang diapakai untuk reccomender system kali ini merupakan model yang sederhana. Arsitektur dari model user dan juga model item sama, yaitu dengan menggunakan Layer embedding yang megubah nilai integer menjadi vektor dengan dimensi tetap(pada kali ini 64).

Task yang kita gunakan untuk melatih model didasarkan pada task Retrieval yang ada pada library tfrs. Task ini akan membuat jarak dari user dan movie yang ia suka dekat, dan dengan movie lain jauh.

In [ ]:
# Mendefinisikan model untuk user dan juga movie
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Mendefinisikan objektif untuk training
def task_retrieval(
    query_embeddings : tf.Tensor,
    candidate_embeddings: tf.Tensor) :

  # score(kemiripan) dari semua user dan item
  scores = tf.linalg.matmul(
        query_embeddings, candidate_embeddings, transpose_b=True)

  num_queries = tf.shape(scores)[0]
  num_candidates = tf.shape(scores)[1]

  # definisi label yang merupakan matirx identitas(user i menyukai item i)
  labels = tf.eye(num_queries, num_candidates)

  # mengunakan loss cross entropy untuk melatih
  loss_function = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.keras.losses.Reduction.SUM)

  loss = loss_function(y_true=labels, y_pred=scores)

  return loss,scores


### Melatih model dan Evaluasi.

Pada blok ini kita melatih model pada MovieLens 100K, Mengevaluasi dan juga memprediksi


In [ ]:
# Membuat model retrieval.
model = MovieLensModel(user_model, movie_model, task_retrieval)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5),run_eagerly=True)

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

In [ ]:
# Mengambil movie yang mempunyai kemiripan paling tinggi

user_idx = np.array(["42"])
user_rep = model.user_model(user_idx)

all_movie_name = [all_item  for all_item in movies.batch(len(list(movies)))][0]
movie_rep = model.movie_model(all_movie_name)

score = tf.linalg.matmul(
        user_rep, movie_rep, transpose_b=True)

index_list = tf.argsort(score, axis=-1, direction='DESCENDING').numpy()
rec_idx = index_list[0,0:3]

print("Top 3 recommendations for user 42: ",all_movie_name.numpy()[rec_idx])